In [1]:
%pip install selenium
%pip install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip install selenium-wire

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 461.9 kB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install python-whois

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 325.8 kB/s eta 0:00:00 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for python-whois: filename=python_whois-0.8.0-py3-none-any.whl size=103247 sha256=5fd184d026ba235f1119d1b4ea5e37566876049d112cb837f229e19f307ac3ff
  Stored in directory: /home/kali/.cache/pip/wheels/8a/d4/1d/bab4b44ad52eadf1b10c5c1ec7cb18a936f24b58bfb95b427e
Successfully built python-whois
Note: you may need to restart the kernel to use updated packages.


In [2]:
from seleniumwire import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import whois
import os

In [4]:
opts = Options()
opts.add_argument("--headless")
opts.add_argument('--no-sandbox')
url = "https://fmoviefree.sc"
directory = f"outputs/{url[8:]}"
os.makedirs(directory, exist_ok=True)
file_path = f"{directory}/whois.txt"
with open(file_path, "w") as file:
    w = whois.whois(url)
    file.write(f"{w.text}")
driver = webdriver.Chrome(options=opts)
driver.get(url)
home = driver.find_element(By.CSS_SELECTOR, "div.hm-button a")
home.click()
movie = driver.find_element(By.XPATH, "//div[@class='ml-item']/a[@title = 'Guardians of the Galaxy Vol. 3']")
movie_name = movie.get_attribute("title")
movie.click()
linking_site_url = driver.find_element(By.CSS_SELECTOR, "div#mv-info a")
linking_site_link = linking_site_url.get_attribute("href")
linking_site_url.click()
time.sleep(10)
file_path = f"{directory}/library_packages.txt"
with open(file_path, "w") as file:
    #print("linking site url: ", linking_site_link, file = file)
    file.write(f"linking site url: {linking_site_link}\n")
    hosting_site_url = driver.find_element(By.CSS_SELECTOR, "iframe#iframe-embed").get_attribute("src")
    #print("hosting site url: ", hosting_site_url, file = file)
    file.write(f"hosting site url: {hosting_site_url}\n")
    script_tags = driver.find_element(By.TAG_NAME, "body").find_elements(By.CSS_SELECTOR, "script[src]")
    link_tags = driver.find_element(By.TAG_NAME, "head").find_elements(By.CSS_SELECTOR, "link[href]")
    script_head_tags = driver.find_element(By.TAG_NAME, "head").find_elements(By.CSS_SELECTOR, "script[src]")
    for script_tag in script_tags:
        src = script_tag.get_attribute("src")
        file.write(f"{src}\n")
    for link_tag in link_tags:
        href = link_tag.get_attribute("href")
        file.write(f"{href}\n")
    for script_head_tag in script_head_tags:
        src = script_head_tag.get_attribute("src")
        file.write(f"{src}\n")
        
file_path = f"{directory}/network_data.txt"
existing_urls = set()
if os.path.isfile(file_path):
    with open(file_path, "r") as file:
        for line in file:
            url = line.split()[0]
            existing_urls.add(url)
else:
    with open(file_path, "w") as file:
        time.sleep(3)
        iframe = driver.find_element(By.XPATH, "//iframe[@id='iframe-embed']")
        driver.switch_to.frame(iframe)
        driver.find_element(By.XPATH,"//div[@aria-label='Start Playback']").click()
        duration = 600
        start_time = time.time()
        while time.time() - start_time < duration:
            for request in driver.requests:
                if request.response and request.url not in existing_urls:
                    file.write(
                        f"{request.url} {request.response.status_code} {request.response.headers['Content-Type']}\n"
                    )
                    existing_urls.add(request.url)
        driver.quit()

KeyboardInterrupt: 